<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/lab3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3강_1: Audio Classification & Tagging
## contents
- audio auto-tagging
---
## audio auto- tagging

<img src="https://storage.googleapis.com/kaggle-media/competitions/freesound/task2_freesound_audio_tagging.png" width="500px" height="300px"></img>
- user의 맥락을 이해할 때 좋은 supervision을 줌
- 일상 속 다양한 소리 중 무엇을 사용할 것인지가 중요
- multi-label인 output이 어떻게 나와야 풀기 쉬울까?

#### 평가
- label-wighted label-ranking average precision(레이블 가중치 레이블 순위 평균 정밀도)
  - 모든 테스트 클립에 대한 오디오 레이블을 예측하는 것으로 구성
  - 각 test data에 대해 관련 레이블의 순위가 매겨진 목록을 검색하는 평균 정밀도 측정(즉, 시스템은 사용 가능한 모든 레이블의 순위를 매기고 순위가 메겨진 목록의 정밀도는 각 실제 레이블까지 평균화)
  - 오직 이 레이블에 달려있는 애만 관심있음

In [ ]:
import numpy as np

def _one_sample_positive_class_precisions(scores, truth):  
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    retrieved_classes = np.argsort(scores)[::-1]
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits

In [ ]:
# 강아지소리/고양이/싸우는 3가지 class 있다고 가정
y_true = np.array([1, 0, 0])  # 강아지 소리에만 labeling
y_score = np.array([0.7, 0.1, 0.2]) # 강아지들이 싸우는 소리 -> 우리는 오직 강아지소리에 대한 label 값만 관심
pos_class_indices, precision_at_hits = _one_sample_positive_class_precisions(y_score, y_true)
print("Correct answer label", pos_class_indices)
print("Score", precision_at_hits)   # 몇 순위인가

Correct answer label [0]
Score [1.]


In [ ]:
y_true = np.array([1, 0, 0])
y_score = np.array([0.1, 0.7, 0.2])
pos_class_indices, precision_at_hits = _one_sample_positive_class_precisions(y_score, y_true)
print("Correct answer label", pos_class_indices)
print("Score", precision_at_hits)

Correct answer label [0]
Score [0.33333333]


#### Step
1. audio representations: 잘 정의된 feature space 생성  
2. feature extraction
3. classifier
<img src="https://image.slidesharecdn.com/talalalsubaie-1220737011220266-9/95/pattern-recognition-26-728.jpg?cb=1220711852" width="500px" height="300px"></img>

## code
[audio auto-tagging_colab](https://colab.research.google.com/drive/1nJ_s50MHWg2t6L3iNJXmMmMhhtA9wjul?usp=sharing#scrollTo=jAItoeNow2wc)